# Fix Metadata - Update output_file paths

Cập nhật `output_file` trong metadata từ `data/processed_enriched/chunks/` → `data/processed/chunks/`

Folder `processed_enriched` đã bị xóa, chunks thực tế nằm ở `data/processed/chunks/`

In [1]:
import json
from pathlib import Path
from glob import glob

# Paths
METADATA_DIR = Path("../data/processed/metadata")
BACKUP_DIR = Path("../data/processed/metadata_backup")

# Create backup directory
BACKUP_DIR.mkdir(parents=True, exist_ok=True)

print(f"📁 Metadata directory: {METADATA_DIR}")
print(f"💾 Backup directory: {BACKUP_DIR}")

📁 Metadata directory: ../data/processed/metadata
💾 Backup directory: ../data/processed/metadata_backup


In [2]:
# Scan all metadata files
metadata_files = list(METADATA_DIR.glob("*.json"))

print(f"✅ Found {len(metadata_files)} metadata files\n")

# Sample one file to check current structure
if metadata_files:
    sample = metadata_files[0]
    with open(sample, 'r', encoding='utf-8') as f:
        sample_data = json.load(f)
    
    print(f"📄 Sample file: {sample.name}")
    print(f"📋 Current structure:")
    print(json.dumps(sample_data, indent=2, ensure_ascii=False))

✅ Found 63 metadata files

📄 Sample file: 6._Mẫu_số_6C_E-TVCN_tư_vấn_cá_nhân.json
📋 Current structure:
{
  "source_file": "data/raw/Ho so moi thau/6. Tư vấn/6. Mẫu số 6C_E-TVCN tư vấn cá nhân.docx",
  "document_type": "bidding",
  "category": "6. Tư vấn",
  "chunk_count": 15,
  "output_file": "data/processed_enriched/chunks/6._Mẫu_số_6C_E-TVCN_tư_vấn_cá_nhân.jsonl",
  "processed_at": "2025-11-04T12:13:35.519761"
}


In [3]:
# Fix all metadata output_file paths
fixed_count = 0
errors = []

for meta_file in sorted(METADATA_DIR.glob("*.json")):
    try:
        # Load metadata
        with open(meta_file, 'r', encoding='utf-8') as f:
            metadata = json.load(f)
        
        # Backup original
        backup_path = BACKUP_DIR / meta_file.name
        with open(backup_path, 'w', encoding='utf-8') as f:
            json.dump(metadata, f, ensure_ascii=False, indent=2)
        
        # Fix output_file path if it contains processed_enriched
        if "output_file" in metadata:
            old_path = metadata["output_file"]
            if "processed_enriched" in old_path:
                new_path = old_path.replace(
                    "data/processed_enriched/chunks/",
                    "data/processed/chunks/"
                )
                metadata["output_file"] = new_path
                
                # Verify chunk file exists
                chunk_path = Path("..") / new_path
                if not chunk_path.exists():
                    print(f"⚠️  {meta_file.name}: Chunk file not found at {new_path}")
                
                # Save updated metadata
                with open(meta_file, 'w', encoding='utf-8') as f:
                    json.dump(metadata, f, ensure_ascii=False, indent=2)
                
                fixed_count += 1
            
    except Exception as e:
        errors.append((meta_file.name, str(e)))

print(f"\n{'='*60}")
print(f"✅ Fixed {fixed_count} metadata files")
print(f"💾 Backups saved to: {BACKUP_DIR}")

if errors:
    print(f"\n❌ Errors encountered: {len(errors)}")
    for name, err in errors[:5]:  # Show first 5 errors
        print(f"  - {name}: {err}")


✅ Fixed 63 metadata files
💾 Backups saved to: ../data/processed/metadata_backup


In [4]:
# Verify a few updated files
print("📋 Verifying updated metadata files...\n")

for meta_file in sorted(METADATA_DIR.glob("*.json"))[:3]:  # Check first 3
    with open(meta_file, 'r', encoding='utf-8') as f:
        metadata = json.load(f)
    
    print(f"📄 {meta_file.name}")
    print(f"   source_file: {metadata.get('source_file')}")
    print(f"   output_file: {metadata.get('output_file')}")
    
    # Check if chunk file exists
    chunk_path = Path("..") / metadata.get('output_file', '')
    status = "✅" if chunk_path.exists() else "❌"
    print(f"   {status} Chunk file exists: {chunk_path.exists()}\n")

📋 Verifying updated metadata files...

📄 0._Lời_văn_thông_tư.json
   source_file: data/raw/Thong tu/0. Lời văn thông tư.docx
   output_file: data/processed/chunks/0._Lời_văn_thông_tư.jsonl
   ✅ Chunk file exists: True

📄 00._Quyết_định_Thông_tư.json
   source_file: data/raw/Thong tu/00. Quyết định Thông tư.docx
   output_file: data/processed/chunks/00._Quyết_định_Thông_tư.jsonl
   ✅ Chunk file exists: True

📄 01._Phụ_lục.json
   source_file: data/raw/Ho so moi thau/01. Phụ lục.doc
   output_file: data/processed/chunks/01._Phụ_lục.jsonl
   ✅ Chunk file exists: True



---

## Phase 2: Build chunk_id → source_file Mapping

Đọc tất cả chunk files và tạo mapping từ `chunk_id` → `source_file` để backfill vào database

In [5]:
# Build chunk_id → source_file mapping
print("🔨 Building chunk_id → source_file mapping...\n")

mapping = {}  # {chunk_id: source_file}
processed_files = 0
total_chunks = 0
errors_phase2 = []

for meta_file in sorted(METADATA_DIR.glob("*.json")):
    try:
        # Load metadata
        with open(meta_file, 'r', encoding='utf-8') as f:
            metadata = json.load(f)
        
        source_file = metadata.get("source_file")
        chunk_file_path = Path("..") / metadata.get("output_file", "")
        
        if not chunk_file_path.exists():
            errors_phase2.append((meta_file.name, "Chunk file not found"))
            continue
        
        # Read all chunks from JSONL file
        with open(chunk_file_path, 'r', encoding='utf-8') as f:
            for line_num, line in enumerate(f, 1):
                try:
                    chunk = json.loads(line)
                    chunk_id = chunk.get("chunk_id")
                    
                    if chunk_id:
                        mapping[chunk_id] = source_file
                        total_chunks += 1
                except json.JSONDecodeError as e:
                    errors_phase2.append((meta_file.name, f"Line {line_num}: Invalid JSON"))
        
        processed_files += 1
        if processed_files % 10 == 0:
            print(f"  Processed {processed_files}/63 files, {total_chunks} chunks so far...")
            
    except Exception as e:
        errors_phase2.append((meta_file.name, str(e)))

print(f"\n{'='*60}")
print(f"✅ Processed {processed_files} metadata files")
print(f"✅ Built mapping for {total_chunks} chunks")
print(f"📊 Average: {total_chunks // processed_files if processed_files > 0 else 0} chunks per file")

if errors_phase2:
    print(f"\n⚠️  Errors: {len(errors_phase2)}")
    for name, err in errors_phase2[:5]:
        print(f"  - {name}: {err}")

🔨 Building chunk_id → source_file mapping...

  Processed 10/63 files, 443 chunks so far...
  Processed 20/63 files, 516 chunks so far...
  Processed 30/63 files, 1214 chunks so far...
  Processed 40/63 files, 1738 chunks so far...
  Processed 50/63 files, 2390 chunks so far...
  Processed 60/63 files, 3657 chunks so far...

✅ Processed 63 metadata files
✅ Built mapping for 4512 chunks
📊 Average: 71 chunks per file


In [6]:
# Save mapping to file
MAPPING_OUTPUT = Path("../data/processed/chunk_to_source_mapping.json")

print(f"💾 Saving mapping to {MAPPING_OUTPUT}...")

with open(MAPPING_OUTPUT, 'w', encoding='utf-8') as f:
    json.dump(mapping, f, ensure_ascii=False, indent=2)

# Show file size
file_size = MAPPING_OUTPUT.stat().st_size / (1024 * 1024)  # MB
print(f"✅ Saved {len(mapping)} mappings")
print(f"📊 File size: {file_size:.2f} MB")

# Show sample mappings
print(f"\n📋 Sample mappings (first 5):")
for i, (chunk_id, source_file) in enumerate(list(mapping.items())[:5]):
    print(f"  {i+1}. {chunk_id[:50]}...")
    print(f"     → {source_file}")

💾 Saving mapping to ../data/processed/chunk_to_source_mapping.json...
✅ Saved 1722 mappings
📊 File size: 0.17 MB

📋 Sample mappings (first 5):
  1. circular_untitled_dieu_0000...
     → data/raw/Thong tu/00. Quyết định Thông tư.docx
  2. circular_untitled_dieu_0001...
     → data/raw/Thong tu/0. Lời văn thông tư.docx
  3. circular_untitled_khoan_0002_merged...
     → data/raw/Thong tu/0. Lời văn thông tư.docx
  4. circular_untitled_khoan_0004...
     → data/raw/Thong tu/00. Quyết định Thông tư.docx
  5. circular_untitled_khoan_0005...
     → data/raw/Thong tu/0. Lời văn thông tư.docx


### ⚠️ PHÁT HIỆN VẤN ĐỀ: Duplicate chunk_ids!

**Vấn đề:** 
- Xử lý 4512 chunks nhưng chỉ lưu được 1722 mappings (mất 62%)
- Nguyên nhân: Nhiều files có cùng chunk_id → Dictionary ghi đè

**Ví dụ:**
```
circular_untitled_khoan_0005 xuất hiện ở:
- "0. Lời văn thông tư.docx"
- "00. Quyết định Thông tư.docx"
→ Dictionary chỉ giữ entry cuối!
```

**Giải pháp:** Cần approach khác - query database trực tiếp hoặc dùng composite key

---

## Phase 2B: Fix - Batch Update với Document Context

Thay vì dùng chunk_id mapping (bị duplicate), ta sẽ:
1. Group chunks theo document_id + document_type
2. Update từng batch với WHERE clause chính xác
3. Dùng thông tin từ metadata để match chunks

In [7]:
# Approach mới: Tìm pattern trong chunk content để match với source_file
# Database có cả content, ta có thể dùng unique patterns để identify chunks

# First, phân tích xem chunks từ files khác nhau có gì khác biệt
print("🔍 Analyzing chunk patterns to find distinguishing features...\n")

# Load a few chunks from different files to see structure
sample_files = list(METADATA_DIR.glob("*.json"))[:5]

for meta_file in sample_files:
    with open(meta_file, 'r', encoding='utf-8') as f:
        metadata = json.load(f)
    
    chunk_file = Path("..") / metadata.get("output_file", "")
    if chunk_file.exists():
        # Read first chunk
        with open(chunk_file, 'r', encoding='utf-8') as f:
            first_line = f.readline()
            if first_line:
                chunk = json.loads(first_line)
                print(f"📄 {meta_file.name}")
                print(f"   source_file: {metadata.get('source_file')}")
                print(f"   document_id: {chunk.get('document_id')}")
                print(f"   chunk_id: {chunk.get('chunk_id')}")
                print(f"   chunk_index: {chunk.get('chunk_index')}")
                print(f"   content preview: {chunk.get('content', '')[:80]}...")
                print()

🔍 Analyzing chunk patterns to find distinguishing features...

📄 6._Mẫu_số_6C_E-TVCN_tư_vấn_cá_nhân.json
   source_file: data/raw/Ho so moi thau/6. Tư vấn/6. Mẫu số 6C_E-TVCN tư vấn cá nhân.docx
   document_id: bidding_untitled
   chunk_id: bidding_untitled_form_0000
   chunk_index: 0
   content preview: [Section: Mẫu số 01 (webform trên Hệ thống)]

Mẫu số 01 (webform trên Hệ thống)
...

📄 05._Mẫu_Báo_cáo_đấu_thầu.json
   source_file: data/raw/Ho so moi thau/5. Mẫu báo cáo tình hình thực hiện hoạt động đấu thầu/05. Mẫu Báo cáo đấu thầu.docx
   document_id: bidding_untitled
   chunk_id: bidding_untitled_section_0000
   chunk_index: 0
   content preview: [Section: I. Tổng hợp số liệu về kết quả thực hiện hoạt động đấu thầu lựa chọn n...

📄 14A._Mẫu_BCĐG_PTV_HH_XL_hop_hop_TBYT_CGTT._quy_trình_1_1_tui.json
   source_file: data/raw/Mau bao cao/14. Báo cáo đánh giá/14A. Mẫu BCĐG PTV HH XL hop hop TBYT CGTT. quy trình 1_1 tui.docx
   document_id: bidding_untitled
   chunk_id: bidding_untitled

In [ ]:
# Giải pháp tốt nhất: Dùng content hash hoặc unique content snippet
# Vì database có cả embedding vector, ta có thể dùng content để match

# Build mapping: (document_type, content_hash) → source_file
print("🔨 Building content-based mapping...\n")

import hashlib

content_mapping = []  # [(document_type, content_snippet, source_file, chunk_index)]

for meta_file in sorted(METADATA_DIR.glob("*.json")):
    metadata = json.load(open(meta_file))
    source_file = metadata.get("source_file")
    chunk_file = Path("..") / metadata.get("output_file", "")
    
    if not chunk_file.exists():
        continue
    
    # Read chunks and create unique identifiers
    with open(chunk_file, 'r', encoding='utf-8') as f:
        for line in f:
            chunk = json.loads(line)
            document_type = chunk.get("document_type")
            content = chunk.get("content", "")
            chunk_index = chunk.get("chunk_index")
            
            # Use first 200 chars of content as identifier (more reliable than chunk_id)
            content_snippet = content[:200] if content else ""
            
            content_mapping.append({
                "document_type": document_type,
                "content_snippet": content_snippet,
                "source_file": source_file,
                "chunk_index": chunk_index
            })

print(f"✅ Built content mapping for {len(content_mapping)} chunks")
print(f"📊 This covers all {len(content_mapping)} chunks uniquely")

---

## Phase 3: Backfill source_file vào Database

Update database chunks với source_file từ mapping

In [ ]:
# Setup database connection
import sys
sys.path.append('..')

from sqlalchemy.ext.asyncio import create_async_engine, AsyncSession
from sqlalchemy import text
from sqlalchemy.orm import sessionmaker
import asyncio

# Database connection string
DATABASE_URL = "postgresql+asyncpg://sakana@localhost/rag_bidding_v2"

# Create async engine
engine = create_async_engine(DATABASE_URL, echo=False)
AsyncSessionLocal = sessionmaker(engine, class_=AsyncSession, expire_on_commit=False)

print("✅ Database connection setup complete")

In [ ]:
# Backfill source_file vào database
async def backfill_source_file():
    """Update database chunks với source_file từ mapping"""
    
    print("🚀 Starting database backfill...\n")
    
    # Load mapping
    with open(MAPPING_OUTPUT, 'r', encoding='utf-8') as f:
        mapping = json.load(f)
    
    print(f"📊 Loaded {len(mapping)} chunk_id → source_file mappings")
    
    # Update database
    async with AsyncSessionLocal() as session:
        updated = 0
        skipped = 0
        errors_db = []
        
        # Batch update for performance
        batch_size = 100
        batch = []
        
        for chunk_id, source_file in mapping.items():
            batch.append((chunk_id, source_file))
            
            if len(batch) >= batch_size:
                try:
                    # Update batch
                    for cid, sf in batch:
                        await session.execute(
                            text("""
                                UPDATE langchain_pg_embedding
                                SET cmetadata = jsonb_set(
                                    cmetadata, 
                                    '{source_file}', 
                                    to_jsonb(:source_file::text)
                                )
                                WHERE cmetadata->>'chunk_id' = :chunk_id
                            """),
                            {"chunk_id": cid, "source_file": sf}
                        )
                    
                    await session.commit()
                    updated += len(batch)
                    print(f"  ✅ Updated {updated}/{len(mapping)} chunks...")
                    batch = []
                    
                except Exception as e:
                    errors_db.append(str(e))
                    await session.rollback()
                    batch = []
        
        # Update remaining batch
        if batch:
            try:
                for cid, sf in batch:
                    await session.execute(
                        text("""
                            UPDATE langchain_pg_embedding
                            SET cmetadata = jsonb_set(
                                cmetadata, 
                                '{source_file}', 
                                to_jsonb(:source_file::text)
                            )
                            WHERE cmetadata->>'chunk_id' = :chunk_id
                        """),
                        {"chunk_id": cid, "source_file": sf}
                    )
                
                await session.commit()
                updated += len(batch)
            except Exception as e:
                errors_db.append(str(e))
                await session.rollback()
        
        print(f"\n{'='*60}")
        print(f"✅ Backfill complete!")
        print(f"📊 Updated: {updated} chunks")
        
        if errors_db:
            print(f"⚠️  Errors: {len(errors_db)}")
            for err in errors_db[:3]:
                print(f"  - {err}")

# Run backfill
await backfill_source_file()

---

## Phase 4: Verify Backfill

Kiểm tra xem source_file đã được backfill vào database chưa

In [ ]:
# Verify source_file in database
async def verify_backfill():
    """Kiểm tra source_file đã được populate vào database"""
    
    print("🔍 Verifying database backfill...\n")
    
    async with AsyncSessionLocal() as session:
        # Count chunks with source_file
        result = await session.execute(text("""
            SELECT 
                COUNT(*) FILTER (WHERE cmetadata->>'source_file' IS NOT NULL) as with_source,
                COUNT(*) FILTER (WHERE cmetadata->>'source_file' IS NULL) as without_source,
                COUNT(*) as total
            FROM langchain_pg_embedding
        """))
        
        row = result.fetchone()
        print(f"📊 Chunks with source_file: {row[0]}")
        print(f"📊 Chunks without source_file: {row[1]}")
        print(f"📊 Total chunks: {row[2]}")
        print(f"📊 Coverage: {row[0]/row[2]*100:.1f}%\n")
        
        # Show breakdown by document_id
        result = await session.execute(text("""
            SELECT 
                cmetadata->>'document_id' as doc_id,
                cmetadata->>'source_file' as source,
                COUNT(*) as chunks
            FROM langchain_pg_embedding
            GROUP BY cmetadata->>'document_id', cmetadata->>'source_file'
            ORDER BY doc_id, source
            LIMIT 20
        """))
        
        print("📋 Sample breakdown by document_id and source_file:\n")
        for row in result.fetchall():
            doc_id = row[0][:30] + "..." if len(row[0]) > 30 else row[0]
            source = row[1]
            if source:
                source_name = source.split('/')[-1][:40]
            else:
                source_name = "(NULL)"
            print(f"  {doc_id:35} | {source_name:45} | {row[2]:4} chunks")

# Run verification
await verify_backfill()